In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import  train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from matplotlib import style
import matplotlib.pyplot as plt
style.use("ggplot")

In [ ]:
iris_datasets = load_iris()
datasets = iris_datasets["data"]
print("the shape of datasets -> ", datasets.shape)
labels = iris_datasets["target"]
print("the shape of leabels -> ", labels.shape)
print("the leabels -> ", labels)
names = iris_datasets["target_names"]
print("the names of iris -> ", names)
feature_names = iris_datasets["feature_names"]
print("the features of iris -> ", feature_names)

In [ ]:
setosa = datasets[:50,:]
versicolor = datasets[50:100,:]
virginica = datasets[100:150,:]

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i in range(4):
    Max = np.max(datasets[:,i])
    Min = np.min(datasets[:,i])
    ax = fig.add_subplot(2,2,i+1)
    ax.hist(setosa[:,i],bins=50, range=(Min,Max),color='blue', label="setosa",stacked=True)
    ax.hist(versicolor[:,i],bins=50, range=(Min,Max),color='green', label="versicolor",stacked=True)
    ax.hist(virginica[:,i],bins=50, range=(Min,Max),color='red', label="virginica",stacked=True)
    ax.set_xlabel("{}".format(feature_names[i]))
    plt.legend()
plt.show()

In [ ]:
Data_setosa = pd.DataFrame(setosa,columns=feature_names)
Data_versicolor = pd.DataFrame(versicolor,columns=feature_names)
Data_virginica = pd.DataFrame(virginica,columns=feature_names)
scatter_matrix(Data_setosa,figsize=(8,8),c="blue",alpha=1)
plt.suptitle("setosa")
scatter_matrix(Data_versicolor,figsize=(8,8),c="blue",alpha=1)
plt.suptitle("versicolor")
scatter_matrix(Data_virginica,figsize=(8,8),c="blue",alpha=1)
plt.suptitle("virginica")
plt.show()

In [ ]:
# for feature1 and feature2
setosa_mean = np.mean(setosa,axis=0)
print("setosa_mean", setosa_mean)
versicolor_mean = np.mean(versicolor,axis=0)
print("versicolor_mean",versicolor_mean)
virginica_mean = np.mean(virginica,axis=0)
print("virginica_mean",virginica_mean)

S_setosa = np.zeros((4,4))
S_versicolor  = np.zeros((4,4))
S_virginica  = np.zeros((4,4))

for i in range(50):
    row_setosa = setosa[i,:].reshape(4,1)
    setosa_mean = setosa_mean.reshape(4,1)
    S_setosa += (row_setosa-setosa_mean).dot((row_setosa-setosa_mean).T)
    
    row_versicolor = versicolor[i,:].reshape(4,1)
    versicolor_mean = versicolor_mean.reshape(4,1)
    S_versicolor += (row_versicolor-versicolor_mean).dot((row_versicolor-versicolor_mean).T)
    
    row_virginica = virginica[i,:].reshape(4,1)
    virginica_mean = virginica_mean.reshape(4,1)
    S_virginica += (row_virginica-virginica_mean).dot((row_virginica-virginica_mean).T)
S_setosa = S_setosa #/ (setosa.shape[0]-1)
print("S_setosa -> ",S_setosa)
S_versicolor  = S_versicolor #/ (versicolor .shape[0]-1)
print("S_versicolor -> ",S_versicolor)
S_virginica  = S_virginica #/ (versicolor .shape[0]-1)
print("S_virginica -> ",S_virginica)

Sw = (S_setosa+S_versicolor+S_virginica) #/3
print("Sw -> ",Sw)

s_mean = setosa_mean.reshape(-1,1)
ve_mean = versicolor_mean.reshape(-1,1)
vi_mean = virginica_mean.reshape(-1,1)
overall_mean = np.mean(datasets,axis=0).reshape(-1,1)
Sb = np.zeros((4,4))
for mean in [s_mean, ve_mean, vi_mean]:
    Sb += 50*((mean-overall_mean).dot((mean-overall_mean).T))
print("Sb -> ",Sb)

# 不是很理解
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))
print("eig_vals -> ",eig_vals)
print("eig_vecs -> ",eig_vecs)
for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(4,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(Sw).dot((Sb)).dot((eigv)),
                                         eig_vals[i]*eigv,
                                         decimal=0, err_msg='', verbose=True)
print('ok')

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = zip(eig_vals, eig_vecs)
# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues

print('Eigenvalues in decreasing order:\n')
for i in eig_pairs:
    print(i[0])
print('Variance explained:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))

In [ ]:
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))
print('Matrix W:\n', W.real)
Datasets = datasets.dot(W)
assert Datasets.shape == (150,2), "The matrix is not 150x2 dimensional."

In [ ]:
plt.scatter(Datasets[:50,0],Datasets[:50,1],marker='o',cmap='red',label="setosa")
plt.scatter(Datasets[50:100,0],Datasets[50:100,1],marker='o',cmap='blue',label="versicolor")
plt.scatter(Datasets[100:150,0],Datasets[100:150,1],marker='o',cmap='green',label="virginica")
plt.xlabel("LDA1")
plt.ylabel("LDA2")
plt.legend()
plt.show()

In [ ]:
plt.scatter(setosa_mean[0],setosa_mean[1],marker='*',cmap='green',s=300)
plt.scatter(setosa[:,0],setosa[:,1],marker='o',cmap='red')
plt.scatter(versicolor_mean[0],versicolor_mean[1],marker='*',cmap='green',s=300)
plt.scatter(versicolor[:,0],versicolor[:,1],marker='o',cmap='blue')
plt.xlabel('sepal length (cm)')
plt.ylabel('sepal width (cm)')
plt.title("setosa & versicolor")
plt.show()

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(datasets, labels,test_size = 0.2)
LDA = LinearDiscriminantAnalysis()
LDA.fit(train_x,train_y)
prediction = LDA.predict(val_x)
print("prediction from LDA -> ",prediction)
print("validation labels -> ",val_y)
# model.score(val_x, val_y) & metrics.accuracy_score(prediction, val_y) -> same effects
accuracy = LDA.score(val_x,val_y)
print("the accuracy -> ",accuracy)
accuracy_score = metrics.accuracy_score(prediction,val_y)
print("the accuracy score -> ",accuracy_score)